1: Importamos Librerías:

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import geopandas as gpd 

2: Cargando base de datos

In [2]:
#Importamos el archivo
empresas_Buc = pd.read_csv('BASE_DE_DATOS_DE_EMPRESAS_CAMARA_DE_COMERCIO_DE_BUCARAMANGA_20240215.csv')

3: Análisis Exploratorio

3.1 Cantidad de registros:

In [3]:

registros = len(empresas_Buc)
print(f""" Hay un total de {registros} registros en el dataset""")

 Hay un total de 83506 registros en el dataset


3.2 Cantidad de variables:

In [4]:
variables = len(empresas_Buc.columns)
print(f""" Hay un total de {variables} columnas en el dataset""")

 Hay un total de 10 columnas en el dataset


3.3 Tipos de variables

In [6]:
empresas_Buc.info()
print('\n')
categoricas = []
numericas = []
for columna in empresas_Buc.columns:
    # Verificar si la columna es categórica y tiene más de 2 niveles
    if empresas_Buc[columna].dtype == 'object' and len(empresas_Buc[columna]) > 1:
        categoricas.append(columna)
    else:
      numericas.append(columna)

print(f""" Con respecto a los tipos de variables tenemos: \n
Hay {len(categoricas)} variables categoricas y son_ {categoricas} \n
Hay {len(numericas)} variables numericas y son: {numericas} """)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83506 entries, 0 to 83505
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NIT                  83487 non-null  object
 1   RAZON_SOCIAL         83506 non-null  object
 2   TIPO_JURIDICO        83506 non-null  object
 3   ESTADO               83506 non-null  object
 4   FECHA_MATRICULA      83506 non-null  object
 5   DESC_CIIU1           83506 non-null  object
 6   DIRECCION_COMERCIAL  83506 non-null  object
 7   CIUDAD               83506 non-null  object
 8   CORREO_ELECTRONICO   83505 non-null  object
 9   TAMAÑO EMPRESA       83506 non-null  object
dtypes: object(10)
memory usage: 6.4+ MB


 Con respecto a los tipos de variables tenemos: 

Hay 10 variables categoricas y son_ ['NIT', 'RAZON_SOCIAL', 'TIPO_JURIDICO', 'ESTADO', 'FECHA_MATRICULA', 'DESC_CIIU1', 'DIRECCION_COMERCIAL', 'CIUDAD', 'CORREO_ELECTRONICO', 'TAMAÑO EMPRESA'] 

Hay 0 variables

3.5 Verificación datos nulos.

In [11]:
registros_Nulos = empresas_Buc.isna().sum()
registros_Nulos

NIT                    19
RAZON_SOCIAL            0
TIPO_JURIDICO           0
ESTADO                  0
FECHA_MATRICULA         0
DESC_CIIU1              0
DIRECCION_COMERCIAL     0
CIUDAD                  0
CORREO_ELECTRONICO      1
TAMAÑO EMPRESA          0
dtype: int64

3.6 Verificación de registros duplicados

In [13]:

duplicated = empresas_Buc[empresas_Buc.duplicated(keep='first', subset='NIT')] #Dataframe de registros duplicados
print(f""" Hay un total de {len(duplicated)} registros duplicados>""")

 Hay un total de 54 registros duplicados>


3.7 Borrado de columnas no apropiadas para modelamiento

Para esto debemos tener a consideración lo siguiente:

- Qué significa cada variable?
- El tipo de dato que representa es acorde a como Pandas lo detecta inicialmente?
- La presencia de null values es indicio de que debería ser eliminada?

In [14]:
empresas_Buc.tail(20)

,NIT,RAZON_SOCIAL,TIPO_JURIDICO,ESTADO,FECHA_MATRICULA,DESC_CIIU1,DIRECCION_COMERCIAL,CIUDAD,CORREO_ELECTRONICO,TAMAÑO EMPRESA
83486,63549216-0,ARIAS CRISTANCHO YOHANA PAOLA,PERSONA NATURAL,CANCELADO,07/14/2023 12:00:00 AM,Comercio Al Por Menor En Establecimientos No E...,Calle 104H # 5A - 52 Barrio PORVENIR,Bucaramanga,pao_1484@hotmail.es,MICROEMPRESA
83487,1097488331-1,GOMEZ MORENO YEIMI ALEJANDRA,PERSONA NATURAL,CANCELADO,07/17/2023 12:00:00 AM,Peluquería Y Otros Tratamientos De Belleza,Calle 56 # 32 - 80 Barrio CABECERA,Bucaramanga,alejagomez0229@gmail.com,MICROEMPRESA
83488,63475247-1,DIAZ ROMERO LUZ ELENA,PERSONA NATURAL,CANCELADO,07/18/2023 12:00:00 AM,Expendio De Bebidas Alcohólicas Para El Consum...,Carrera 7 # 8 - 00 Barrio CENTRO,Piedecuesta,luhediro@gmail.com,MICROEMPRESA
83489,1095298859-5,RINCON AMAYA DARCY MAYLI,PERSONA NATURAL,CANCELADO,07/19/2023 12:00:00 AM,Comercio Al Por Menor De Electrodomésticos Y G...,Calle 45 # 19 - 66,Bucaramanga,sillasamayala45@gmail.com,MICROEMPRESA
83490,1127345003-5,CRUZ PARRA JHON EUDEL,PERSONA NATURAL,CANCELADO,07/19/2023 12:00:00 AM,Publicidad,Calle 100 # 33A - 14 Apartamento 101,Floridablanca,jhoncruzseo@gmail.com,MICROEMPRESA
83491,1062912967-7,HOYOS ARCE MAIRA LUCIA,PERSONA NATURAL,CANCELADO,07/24/2023 12:00:00 AM,"Comercio De Partes, Piezas (Autopartes) Y Acce...",ANILLO VIAL Kilómetro 3 Vereda RIO FRIO Parque...,Giron,mairaluciahoyosarce@gmail.com,MICROEMPRESA
83492,77025456-3,LARROTA LIZARAZO EDUARDO,PERSONA NATURAL,CANCELADO,07/25/2023 12:00:00 AM,Comercio Al Por Menor En Establecimientos No E...,Carrera 15 # 34 - 49 Local 101A Barrio,Bucaramanga,johan_2274@gmail.com,MICROEMPRESA
83493,1100951055-9,HERNANDEZ VELANDIA BETSSY JOHANA,PERSONA NATURAL,CANCELADO,07/26/2023 12:00:00 AM,Comercio Al Por Menor De Otros Artículos Domés...,CL 29 B # 13 - 12 Barrio LA LIBERTAD,San Gil,betssyhv20@gmail.com,MICROEMPRESA
83494,1101694354-7,ALBA ACUÑA CARLOS ANDRES,PERSONA NATURAL,CANCELADO,07/31/2023 12:00:00 AM,"Comercio Al Por Mayor De Desperdicios, Desecho...",Calle 11 # 20 - 76 Barrio COMUNEROS,Socorro,albaacunacarlosandres@gmail.com,MICROEMPRESA
83495,5685047-3,ECHEVERRI ROJAS JUAN JOSE,PERSONA NATURAL,CANCELADO,08/01/2023 12:00:00 AM,Comercio Al Por Menor De Carnes (Incluye Aves ...,Carrera 15 PEATONAL 14B - 05 Barrio PUERTO M...,Giron,juanjoseecheverri2@gmail.com,MICROEMPRESA


In [15]:
duplicated

,NIT,RAZON_SOCIAL,TIPO_JURIDICO,ESTADO,FECHA_MATRICULA,DESC_CIIU1,DIRECCION_COMERCIAL,CIUDAD,CORREO_ELECTRONICO,TAMAÑO EMPRESA
71001,0,GESTION DE PROPIEDAD HORIZONTAL CH SAS,SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS,ACTIVO,01/06/2023 12:00:00 AM,Actividades Inmobiliarias Realizadas A Cambio ...,Avenida 87 # 24 - 09 Torre 2 Apartamento 703,Bucaramanga,grupopropiedadhorizontalch@gmail.com,MICROEMPRESA
71772,0,PREXON S.A.S.,SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS,ACTIVO,01/23/2023 12:00:00 AM,Actividades De Desarrollo De Sistemas Informát...,Calle 197 # 15 - 425 Casa 23,Floridablanca,danieldavid2001@gmail.com,MICROEMPRESA
71780,0,INVERSIONES PRIETO FLOREZ S.A.S,SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS,ACTIVO,01/23/2023 12:00:00 AM,Actividades De Administración Empresarial,Carrera 20 # 29 - 112 Casa 24 Conjunto RESIDE...,Floridablanca,gcomercialambientast@gmail.com,MICROEMPRESA
71936,0,TRANSPORTE Y LOGÍSTICA D&D S.A.S.,SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS,ACTIVO,01/25/2023 12:00:00 AM,Transporte De Carga Por Carretera,Carrera 20 # 61 - 58,Floridablanca,transporteylogisticadydsas@hotmail.com,MICROEMPRESA
72039,0,LOS PEÑAS SABOR COSTEÑO S.A.S,SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS,ACTIVO,01/26/2023 12:00:00 AM,"Comercio Al Por Menor De Leche, Productos Láct...",Carrera 16 # 45 - 138,Bucaramanga,analucia0280@hotmail.com,MICROEMPRESA
72088,0,SATA CONSTRUCCIONES S.A.S.,SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS,ACTIVO,01/27/2023 12:00:00 AM,Comercio Al Por Menor De Artículos De Ferreter...,Calle 97 # 33 - 60,Bucaramanga,yaksonsuarez@hotmail.com,MICROEMPRESA
72284,NaN,"""COMERCIALIZADORA FERRELOGISTICA S.A.S.""",SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS,ACTIVO,01/31/2023 12:00:00 AM,Comercio Al Por Menor De Artículos De Ferreter...,Calle 58 # 8 - 51,Giron,vagaskarina1215@gmail.com,MICROEMPRESA
72574,0,CONSTRUCTORA E INMOBILIARIA H&Q S.A.S.,SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS,ACTIVO,02/06/2023 12:00:00 AM,Construcción De Edificios Residenciales,Calle 52 # 23 - 42,Bucaramanga,paulasofia2017@outlook.es,MICROEMPRESA
73658,NaN,CALZADO S'MANTILLA S.A.S.,SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS,ACTIVO,02/24/2023 12:00:00 AM,"Fabricación De Otros Tipos De Calzado, Excepto...",Calle 31 # 1 - 96 Barrio LA CUMBRE,Floridablanca,calzamantilla@gmail.com,MICROEMPRESA
73732,0,TRANSPORTE Y SERVICIOS METROPOLITANOS DE COLOM...,SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS,ACTIVO,02/27/2023 12:00:00 AM,Actividades De Mensajería,Calle 14 # 12 - 20 Barrio GAITAN,Bucaramanga,juandiego_pineda@hotmail.com,MICROEMPRESA
